In [38]:
import os
import cv2
import numpy as np

In [39]:
def read_data(csv_path):
    with open(csv_path, "r") as file:
        lines = file.read().splitlines()
    return [ line.split(",") for line in lines ]

In [40]:
train_data = read_data("train.csv")
test_data = read_data("test.csv")

In [41]:
def read_and_resize(path, size):
    bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return cv2.resize(rgb, size)

In [42]:
def load_data(data):
    image_list = []
    label_list = []
    for path, name in data:
        image = read_and_resize(path, (32, 32))
        name = int(name)
        image_list.append(image)
        label_list.append(name)
    return np.stack(image_list, axis=0), np.array(label_list)

In [43]:
x_train, y_train = load_data(train_data)
x_test, y_test = load_data(test_data)

In [44]:
def label2onehot(label):
    return np.eye(2)[label]

In [45]:
y_train_onehot = label2onehot(y_train)
y_test_onehot = label2onehot(y_test)

In [46]:
 x_train = x_train /255
x_test = x_test /255

In [47]:
 print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(3160, 32, 32, 3) (3160,)
(789, 32, 32, 3) (789,)


## Model

In [48]:
from tensorflow import keras

In [49]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(keras.layers.Activation(keras.activations.relu))

model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256))
model.add(keras.layers.Activation(keras.activations.linear))
model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation(keras.activations.linear))
model.add(keras.layers.Dense(2))
model.add(keras.layers.Activation(keras.activations.softmax))

loss_func = keras.losses.categorical_crossentropy
metrics = [
    keras.metrics.categorical_accuracy
]
optimizer = keras.optimizers.Adam(lr=0.0001)

model.compile(loss=loss_func, optimizer=optimizer, metrics=metrics)

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_21 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 16, 16, 32)       

## Train

In [50]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", 
        monitor='val_loss', 
        mode='min',
        save_best_only=True
    ),
]

In [51]:
his = model.fit(
    x_train,
    y_train_onehot,
    validation_data=(x_test, y_test_onehot),
    epochs=30,
    batch_size=32,
    verbose=1,
    callbacks=callbacks
)

Train on 3160 samples, validate on 789 samples
Epoch 1/30
3160/3160 [==============================] - 7s 2ms/sample - loss: 0.2767 - categorical_accuracy: 0.8699 - val_loss: 0.1429 - val_categorical_accuracy: 0.9544
Epoch 2/30
3160/3160 [==============================] - 6s 2ms/sample - loss: 0.1189 - categorical_accuracy: 0.9563 - val_loss: 0.1063 - val_categorical_accuracy: 0.9708
Epoch 3/30
3160/3160 [==============================] - 6s 2ms/sample - loss: 0.0952 - categorical_accuracy: 0.9671 - val_loss: 0.0991 - val_categorical_accuracy: 0.9696
Epoch 4/30
3160/3160 [==============================] - 6s 2ms/sample - loss: 0.0901 - categorical_accuracy: 0.9680 - val_loss: 0.0953 - val_categorical_accuracy: 0.9734
Epoch 5/30
3160/3160 [==============================] - 6s 2ms/sample - loss: 0.0820 - categorical_accuracy: 0.9728 - val_loss: 0.0965 - val_categorical_accuracy: 0.9785
Epoch 6/30
3160/3160 [==============================] - 6s 2ms/sample - loss: 0.0770 - categorical_accu

In [53]:
val_loss, val_acc = model.evaluate(x_test,y_test_onehot)

789/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================